In [1]:
import gdi_loader as gdi
from sklearn import svm
import bert_swiss_lm as bsl
import itertools
import torch
from tqdm import tqdm
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report

In [3]:
loader = gdi.GdiLoader(set_path="/Users/raphael/Downloads/gdi-vardial-2019/")
bert_embedder = bsl.BertSwissLm("/Users/raphael/Downloads/bert-swiss-lm")
#train_data = loader.create_dataframe(which_type="train")
#dev_data = loader.create_dataframe(which_type="dev")
test_data = loader.create_dataframe(which_type="gold")

Some weights of the model checkpoint at /Users/raphael/Downloads/bert-swiss-lm were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [13]:
#n_data = train_data[0:100]
#dev_df = loader.create_dataframe("dev")

In [14]:
import time
start = time.time()
train_embedding_df = bert_embedder.add_sentence_embedding(train_data)
print(time.time() - start)

2141.757797718048


In [5]:
gold_embedding_df = bert_embedder.add_sentence_embedding_fast(test_data)

In [7]:
gold_embedding_df.to_feather("test_embedding_bert_swiss_lm.feather")

In [17]:
start = time.time()
dev_embedding_df = bert_embedder.add_sentence_embedding(dev_data)
print(time.time() - start)

563.8175148963928


In [18]:
dev_embedding_df.to_feather("dev_embedding_bert_swiss_lm.feather")

In [16]:
train_embedding_df.to_feather("train_embedding_bert_swiss_lm.feather")

In [3]:
sentences_train = [text for text, lable in itertools.islice(loader.get_next(), 1000)]
lables_train = [lable for text, lable in itertools.islice(loader.get_next(), 1000)]
sentences_dev = [text for text, lable in itertools.islice(loader_dev.get_next(), 1000)]
lables_dev = [lable for text, lable in itertools.islice(loader_dev.get_next(), 1000)]

In [4]:
embeddings = [bert_embedder.get_sentence_embedding(sentence) for sentence in tqdm(sentences_train)]
embeddings_dev = [bert_embedder.get_sentence_embedding(sentence) for sentence in tqdm(sentences_dev)]

100%|██████████| 1000/1000 [02:12<00:00,  7.56it/s]


In [5]:
embedding_tensor = torch.cat(embeddings, dim=0)
embdedings_numpy = embedding_tensor.numpy()
embedding_tensor_dev = torch.cat(embeddings_dev, dim=0)
embdedings_numpy_dev = embedding_tensor_dev.numpy()

In [6]:
clf = svm.SVC()
clf.fit(embdedings_numpy, lables_train)

SVC()

In [7]:
print(clf.predict(embdedings_numpy))
print(lables_train)

['BS' 'BS' 'BS' 'LU' 'ZH' 'BE' 'BE' 'ZH' 'LU' 'BS' 'ZH' 'BE' 'ZH' 'BE'
 'ZH' 'LU' 'ZH' 'BE' 'BE' 'ZH' 'BE' 'ZH' 'BE' 'BS' 'BS' 'BE' 'BS' 'LU'
 'ZH' 'LU' 'LU' 'BE' 'ZH' 'ZH' 'BS' 'ZH' 'ZH' 'ZH' 'LU' 'LU' 'BS' 'ZH'
 'LU' 'BE' 'LU' 'BE' 'LU' 'BE' 'BS' 'BS' 'ZH' 'BE' 'ZH' 'ZH' 'ZH' 'ZH'
 'BS' 'LU' 'LU' 'LU' 'LU' 'ZH' 'ZH' 'ZH' 'BE' 'ZH' 'ZH' 'BE' 'BE' 'LU'
 'BE' 'ZH' 'LU' 'BE' 'LU' 'BE' 'BS' 'ZH' 'BE' 'ZH' 'LU' 'BE' 'BS' 'LU'
 'LU' 'LU' 'LU' 'ZH' 'BS' 'BS' 'BE' 'ZH' 'BS' 'BS' 'BE' 'BE' 'ZH' 'BE'
 'BE' 'BE' 'ZH' 'ZH' 'ZH' 'BE' 'BE' 'BS' 'ZH' 'BE' 'BE' 'LU' 'BS' 'BS'
 'BE' 'BE' 'LU' 'BE' 'ZH' 'LU' 'ZH' 'BE' 'LU' 'ZH' 'LU' 'BS' 'BE' 'BE'
 'BS' 'LU' 'BS' 'LU' 'LU' 'BE' 'ZH' 'ZH' 'BS' 'ZH' 'BE' 'BE' 'ZH' 'ZH'
 'ZH' 'BE' 'BS' 'ZH' 'BE' 'BE' 'ZH' 'BE' 'ZH' 'ZH' 'BS' 'LU' 'BE' 'BS'
 'BE' 'BE' 'BE' 'BE' 'BE' 'BE' 'BE' 'ZH' 'BE' 'ZH' 'BS' 'ZH' 'ZH' 'LU'
 'BS' 'ZH' 'ZH' 'LU' 'LU' 'ZH' 'LU' 'ZH' 'BE' 'BE' 'ZH' 'ZH' 'BE' 'BE'
 'LU' 'BE' 'ZH' 'LU' 'BE' 'LU' 'BS' 'LU' 'BE' 'ZH' 'ZH' 'ZH' 'ZH' 'ZH'
 'BS' 

In [8]:
prediction_dev = clf.predict(embdedings_numpy_dev)
print(prediction_dev)
print(lables_dev)

['LU' 'BS' 'LU' 'LU' 'BE' 'ZH' 'ZH' 'LU' 'BS' 'LU' 'LU' 'BE' 'ZH' 'LU'
 'LU' 'BE' 'LU' 'BE' 'LU' 'LU' 'BS' 'BE' 'BS' 'ZH' 'BS' 'BE' 'ZH' 'BS'
 'BE' 'BE' 'BS' 'BS' 'BS' 'LU' 'ZH' 'BS' 'ZH' 'ZH' 'LU' 'LU' 'LU' 'ZH'
 'ZH' 'BS' 'BE' 'ZH' 'LU' 'BS' 'LU' 'ZH' 'BE' 'ZH' 'BS' 'ZH' 'BS' 'ZH'
 'ZH' 'BE' 'BE' 'BS' 'LU' 'BS' 'BE' 'ZH' 'ZH' 'ZH' 'LU' 'BS' 'BS' 'LU'
 'BE' 'BS' 'ZH' 'ZH' 'ZH' 'ZH' 'BE' 'BE' 'BS' 'ZH' 'ZH' 'BE' 'BS' 'ZH'
 'BS' 'BS' 'BS' 'BS' 'BE' 'BS' 'ZH' 'BS' 'ZH' 'ZH' 'LU' 'ZH' 'BS' 'ZH'
 'BS' 'ZH' 'BS' 'BS' 'LU' 'BS' 'BE' 'BS' 'LU' 'ZH' 'ZH' 'BS' 'BE' 'LU'
 'ZH' 'ZH' 'ZH' 'BE' 'BE' 'ZH' 'BS' 'ZH' 'BE' 'LU' 'BS' 'BE' 'BS' 'BS'
 'BE' 'BS' 'BS' 'BS' 'LU' 'BE' 'ZH' 'LU' 'LU' 'BE' 'BE' 'LU' 'BE' 'ZH'
 'BS' 'BE' 'BS' 'BS' 'BE' 'ZH' 'ZH' 'ZH' 'ZH' 'BE' 'BS' 'BS' 'ZH' 'ZH'
 'LU' 'ZH' 'ZH' 'BS' 'BS' 'BS' 'BS' 'LU' 'BE' 'LU' 'BE' 'BS' 'BE' 'BE'
 'ZH' 'ZH' 'LU' 'ZH' 'ZH' 'ZH' 'ZH' 'BE' 'BS' 'BE' 'BS' 'LU' 'ZH' 'LU'
 'BE' 'BS' 'ZH' 'LU' 'BS' 'BE' 'BE' 'ZH' 'BE' 'BS' 'BE' 'BS' 'BS' 'ZH'
 'BE' 

In [13]:
print(precision_score(lables_dev, prediction_dev, average="macro"))
print(classification_report(lables_dev, prediction_dev))

0.6147505478251539
              precision    recall  f1-score   support

          BE       0.60      0.62      0.61       242
          BS       0.69      0.52      0.60       335
          LU       0.69      0.49      0.57       235
          ZH       0.48      0.84      0.61       188

    accuracy                           0.60      1000
   macro avg       0.61      0.62      0.60      1000
weighted avg       0.63      0.60      0.60      1000



In [10]:
loader = gdi.GdiLoader(set_path="/Users/raphael/Downloads/gdi-vardial-2019/")
bert_embedder = bsl.BertSwissLm("ZurichNLP/swissbert")
bert_embedder.model.set_default_language("de_CH")


Some weights of the model checkpoint at ZurichNLP/swissbert were not used when initializing XmodModel: ['lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.bias']
- This IS expected if you are initializing XmodModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XmodModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XmodModel were not initialized from the model checkpoint at ZurichNLP/swissbert and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [20]:

train_data_2 = loader.create_dataframe(which_type="train")
dev_data_2 = loader.create_dataframe(which_type="dev")
test_data_2 = loader.create_dataframe(which_type="gold")

In [23]:
import time
start = time.time()
train_embedding_df = bert_embedder.add_sentence_embedding(train_data_2)
print(time.time() - start)

2436.4702048301697


In [24]:
train_embedding_df.to_feather("train_embedding_swissbert.feather")

In [26]:
import time
start = time.time()
test_embedding_df = bert_embedder.add_sentence_embedding(test_data_2)
print(time.time() - start)

819.1357779502869


In [27]:
test_embedding_df.to_feather("test_embedding_swissbert.feather")
